In [ ]:
#INSTALACIÓN DE PAQUETES E IMPORTACION DE LIBRERÍAS
from google.colab import drive
import os
import fiftyone as fo
import fiftyone.zoo as foz
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from fiftyone import ViewField as F
import fiftyone.core.utils as fou
from collections import Counter
from fiftyone.core.stages import F
import fiftyone.brain as fob
import shutil
import cv2
import pandas as pd
from dask import bag, diagnostics
from urllib import request
from glob import glob
import matplotlib.pyplot as plt
import missingno as msno
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import img_to_array, array_to_img, load_img
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split
import keras
from tensorflow.keras.utils import to_categorical
import shutil

class Conexion:
    def __init__(self):
        drive.mount('/content/gdrive')
        self.dataset = fo.Dataset.from_images_dir("/content/gdrive/MyDrive/DATASET/mydataset")
        
    def crearmodelo(self):
        #CREACIÓN DEL MODELO 
        self.model = foz.load_zoo_model("mobilenet-v2-imagenet-torch")

        #PROCESO DE EMBEDDINGS
        self.embeddings = self.dataset.compute_embeddings(self.model)

        print(self.embeddings.shape)

        #SIMILARYTY MATRIX
        self.similarity_matrix = cosine_similarity(self.embeddings)

        print(self.similarity_matrix.shape)
        print(self.similarity_matrix)

        n = len(self.similarity_matrix)

        self.similarity_matrix = self.similarity_matrix - np.identity(n)

        id_map = [s.id for s in self.dataset.select_fields(["id"])]

        #CREACIÓN DEL SAMPLE MAX_SIMILARITY
        for idx, self.sample in enumerate(self.dataset):
            self.sample["max_similarity"] = self.similarity_matrix[idx].max()
            self.sample.save()

    def elimimaduplicados(self):
        self.dataset.match(F("max_similarity")>0.98)
        id_map = [s.id for s in self.dataset.select_fields(["id"])] 

        #ELIMINAR DUPLICADOS
        thresh = 0.98
        self.samples_to_remove = set()
        self.samples_to_keep = set()

        for idx, self.sample in enumerate(self.dataset):
            if self.sample.id not in self.samples_to_remove:
                # Keep the first instance of two duplicates
                self.samples_to_keep.add(self.sample.id)
                
                self.dup_idxs = np.where(self.similarity_matrix[idx] > thresh)[0]
                for self.dup in self.dup_idxs:
                    # We kept the first instance so remove all other duplicates
                    self.samples_to_remove.add(id_map[self.dup])

                if len(self.dup_idxs) > 0:
                    self.sample.tags.append("has_duplicates")
                    self.sample.save()
            else:
                self.sample.tags.append("duplicate")
                self.sample.save()

        #EVALUACIÓN RÁPIDA DE IMÁGENES QUE COINCIDEN 
        #Iterate over the samples and compute their file hashes:

        for self.sample in self.dataset:
            self.sample["file_hash"] = fou.compute_filehash(self.sample.filepath)
            self.sample.save()

        print(self.dataset)

        filehash_counts = Counter(self.sample.file_hash for self.sample in self.dataset)
        dup_filehashes = [k for k, v in filehash_counts.items() if v > 1]

        print("Number of duplicate file hashes: %d" % len(dup_filehashes))

        dup_view = (self.dataset
            # Extract samples with duplicate file hashes
            .match(F("file_hash").is_in(dup_filehashes))
            # Sort by file hash so duplicates will be adjacent
            .sort_by("file_hash")
        )

        print("Number of images that have a duplicate: %d" % len(dup_view))
        print("Number of duplicates: %d" % (len(dup_view) - len(dup_filehashes)))

        print("Length of dataset before: %d" % len(self.dataset))

        _dup_filehashes = set()
        for self.sample in dup_view:
            if self.sample.file_hash not in _dup_filehashes:
                _dup_filehashes.add(self.sample.file_hash)
                continue

            del self.dataset[self.sample.id]

        print("Length of dataset after: %d" % len(self.dataset))

        # Verify that the dataset no longer contains any duplicates
        print("Number of unique file hashes: %d" % len({s.file_hash for s in self.dataset}))

        self.dataset.delete_samples(list(self.samples_to_remove))
        print("Luego de evaluar max_similarity: %d" % len(self.dataset))

        
        # evaluar la uniqueness de la data
        fob.compute_uniqueness(self.dataset)

        # Sort in decreasing order of uniqueness (least unique at the end)
        #de mayor a menor
        similar_ARMA = self.dataset.sort_by("uniqueness", reverse=True)

        # Verify that the most unique sample has the maximal uniqueness of 1.0
        #Se crea el array uniqueness_fine para almacenar los valores de uniqueness
        uniqueness_fine = [self.sample.uniqueness for self.sample in self.dataset]
        print(uniqueness_fine)
        p=len(uniqueness_fine) 
        print (p)

        #Se crea el array dup_uniqueness_fine para almacenar los valores de uniqueness > c
        dup_uniqueness_fine = []
        dup_uniqueness_bad = []
        c = 0.2  #valor de referencia para ver cuanta unicidad minima es necesaria
        for v in uniqueness_fine:
          if v >= c:
            dup_uniqueness_fine.append(v)
            
          if v < c:
            dup_uniqueness_bad.append(v) #data uniqueness < 0.2

        n=len(dup_uniqueness_fine)
        m=len(dup_uniqueness_bad)
        print("Número total de archivos al inicio:" + str(p))
        f=(n/p)*100
        print("Number of files wich uniqueness is > 0.2: %d" % n)
        print("Percent of data that is fine > 0.2: %d" % f )

        #Se crea el array dup_uniqueness_fine_data para 
        #pasar de los valores de uniqueness a los nombres de las fotos
        ten_best = [x.filepath for x in similar_ARMA.limit(n)]
        dup_uniqueness_fine_data = []

        for filepath in ten_best:
            dup_uniqueness_fine_data.append(filepath.split('/')[-1])
        print(dup_uniqueness_fine_data)

        # Sort in decreasing order of uniqueness (least unique at the end)
        #de menor a mayor
        similar_ARMA = self.dataset.sort_by("uniqueness", reverse=False)

        #Se crea el array dup_uniqueness_bad_data para 
        #pasar de los valores de uniqueness a los nombres de las fotos
        ten_best = [x.filepath for x in similar_ARMA.limit(m)]
        dup_uniqueness_bad_data = []

        for filepath in ten_best:
            dup_uniqueness_bad_data.append(filepath.split('/')[-1])
        print(dup_uniqueness_bad_data)

        #Crear carpeta para copiar datos unicos
        try:
          # Directory
          directory = 'UNIQUENESS_FINE_DATA'
          print(directory)  
          # Parent Directory path
          parent_dir = "/content/gdrive/MyDrive/DATASET"
          path = os.path.join(parent_dir, directory)
          print(path)
          os.mkdir(path)
        except:
          print("Error, la carpeta no se puede crear o ya existe" )

        #Crear carpeta para copiar datos NO unicos
        try:
          # Directory
          directory = 'UNIQUENESS_BAD_DATA'
          print(directory)  
          # Parent Directory path
          parent_dir = "/content/gdrive/MyDrive/DATASET"
          path = os.path.join(parent_dir, directory)
          print(path)
          os.mkdir(path)
        except:
          print("Error, la carpeta no se puede crear o ya existe" )

        #SE QUITA LA DATA DE LA DATASET HASTA QUE EL %DE IMÁGENES CUYO UNIQUENESS SEA <0.2 SEA MENOR A 10% 
        #Y SE ENVÍA ESTA DATA A LA CARPETA UNIQUENESS_FINE_DATA 
        if (f < 90):
          img_index =len(dup_uniqueness_bad_data)
          while (img_index >= 0):
            frame_name =dup_uniqueness_bad_data[img_index-1]
            src_path = '/content/gdrive/MyDrive/DATASET/mydataset/' + frame_name #CARPETA DE ORIGEN
            dst_path = '/content/gdrive/MyDrive/DATASET/UNIQUENESS_BAD_DATA/' + frame_name #CARPETA DE FIN
            try:
              shutil.move(src_path, dst_path)
            except:
              print("Verificar si se copió: " + dup_uniqueness_bad_data[img_index-1])
            img_index -= 1
       
        #SI EL %DE IMÁGENES CUYO UNIQUENESS ES <0.2 ES MENOR A 10% SE MUEVE LA DATA
        if (f > 90):
          img_index =len(dup_uniqueness_fine_data)
          while (img_index >= 0):
            frame_name =dup_uniqueness_fine_data[img_index-1]
            src_path = '/content/gdrive/MyDrive/DATASET/mydataset/' + frame_name #CARPETA DE ORIGEN
            dst_path = '/content/gdrive/MyDrive/DATASET/UNIQUENESS_FINE_DATA/' + frame_name #CARPETA DE FIN
            try:
              shutil.move(src_path, dst_path)
            except:
              print("Verificar si se copió: " + dup_uniqueness_fine_data[img_index-1])
            img_index -= 1

        j=1
        #SI EL %DE IMÁGENES CUYO UNIQUENESS ES <0.2 ES MAYOR A 10% 
        while (f < 90):

          fob.compute_uniqueness(self.dataset)

          #Se crea el array uniqueness_fine para almacenar los valores de uniqueness
          uniqueness_fine = [self.sample.uniqueness for self.sample in self.dataset]
          print(uniqueness_fine)
          p=len(uniqueness_fine) 
          print(p)

          #Se crea el array dup_uniqueness_fine para almacenar los valores de uniqueness > c
          dup_uniqueness_fine = []
          dup_uniqueness_bad = []
          c = 0.2  #valor de referencia para ver cuanta unicidad minima es necesaria

          for v in uniqueness_fine:
            if v >= c:
              dup_uniqueness_fine.append(v)
            if v < c:
              dup_uniqueness_bad.append(v) #data uniqueness < 0.2

          n=len(dup_uniqueness_fine)
          m=len(dup_uniqueness_bad)
          print("Número total de archivos al inicio:" + str(p))
          f=(n/p)*100
          print("Number of files wich uniqueness is > 0.2: %d" % n)
          print("Percent of data that is fine > 0.2: %d" % f )

          similar_ARMA = self.dataset.sort_by("uniqueness", reverse=True)
          #Se crea el array dup_uniqueness_fine_data para 
          #pasar de los valores de uniqueness a los nombres de las fotos
          ten_best = [x.filepath for x in similar_ARMA.limit(n)]
          dup_uniqueness_fine_data = []

          for filepath in ten_best:
              dup_uniqueness_fine_data.append(filepath.split('/')[-1])
          print(dup_uniqueness_fine_data)

          # Sort in decreasing order of uniqueness (least unique at the end)
          #de menor a mayor
          similar_ARMA = self.dataset.sort_by("uniqueness", reverse=False)

          #Se crea el array dup_uniqueness_bad_data para 
          #pasar de los valores de uniqueness a los nombres de las fotos
          ten_best = [x.filepath for x in similar_ARMA.limit(m)]
          dup_uniqueness_bad_data = []

          for filepath in ten_best:
              dup_uniqueness_bad_data.append(filepath.split('/')[-1])
          print(dup_uniqueness_bad_data)

          #SE QUITA LA DATA DE LA DATASET HASTA QUE EL %DE IMÁGENES CUYO UNIQUENESS SEA <0.2 SEA MENOR A 10% 
          #Y SE ENVÍA ESTA DATA A LA CARPETA UNIQUENESS_FINE_DATA 
          if (f < 90):
            img_index =len(dup_uniqueness_bad_data)
            while (img_index >= 0):
              frame_name =dup_uniqueness_bad_data[img_index-1]
              src_path = '/content/gdrive/MyDrive/DATASET/mydataset/' + frame_name #CARPETA DE ORIGEN
              dst_path = '/content/gdrive/MyDrive/DATASET/UNIQUENESS_BAD_DATA/' + frame_name #CARPETA DE FIN
              try:
                shutil.move(src_path, dst_path)
              except:
                print("Verificar si se copió: " + dup_uniqueness_bad_data[img_index-1])
              img_index -= 1
        
          #SI EL %DE IMÁGENES CUYO UNIQUENESS ES <0.2 ES MENOR A 10% SE MUEVE LA DATA
          if (f > 90):
            img_index =len(dup_uniqueness_fine_data)
            while (img_index >= 0):
              frame_name =dup_uniqueness_fine_data[img_index-1]
              src_path = '/content/gdrive/MyDrive/DATASET/mydataset/' + frame_name #CARPETA DE ORIGEN
              dst_path = '/content/gdrive/MyDrive/DATASET/UNIQUENESS_FINE_DATA/' + frame_name #CARPETA DE FIN
              try:
                shutil.move(src_path, dst_path)
              except:
                print("Verificar si se copió: " + dup_uniqueness_fine_data[img_index-1])
              img_index -= 1

          self.dataset.match(F("uniqueness")<0.2)
          id_map = [s.id for s in self.dataset.select_fields(["id"])] 

          #ELIMINAR DUPLICADOS
          thresh = 0.2
          self.samples_to_remove_u = set()
          self.dup_idxsu = np.where(similar_ARMA[idx] < thresh)[0]
          for self.dupi in self.dup_idxsu:
            self.samples_to_remove_u.add(id_map[self.dupi])
          self.dataset.delete_samples(list(self.samples_to_remove_u))
          print("Luego de evaluar uniqueness: %d" % len(self.dataset))

                
        

    def preprocesardata(self,img_size):
        ####Este código es para obtener la cantidad de imagenes en el dataset y el conjunto de datos que representa cada imagen
        #150 img_size
        ruta_armas = "/content/gdrive/MyDrive/DATASET/UNIQUENESS_FINE_DATA"
        armas_training=[]

        for img in os.listdir(ruta_armas):
          img = cv2.imread(os.path.join(ruta_armas,img))
          img_gray = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
          img_gray_resize = cv2.resize(img_gray,(img_size,img_size))
          armas_training.append([img_gray_resize])

        #Este código es para revisar mediante visualizacion las imagenes dentro del dataset, es decir que  determinada posición coincide con lo dispuesto en mi datased
        plt.figure()
        plt.imshow(np.squeeze(armas_training[2]))
        plt.colorbar()
        plt.grid(False)
        plt.show()  

        #ESTE CODIGO ES PARA COPIAR LA DATASET HACIA OTRA CARPETA (SE REPITE POR CADA TIPO DE ARMA CON SU PROPIA CARPETA)
        #ARMAS BLANCAS
        ##########################

        source_dir = "/content/gdrive/MyDrive/DATASET/UNIQUENESS_FINE_DATA"
        destination_dir = "/content/gdrive/MyDrive/DATASET/carpetaAB"
        shutil.copytree(source_dir, destination_dir)

        #ESTE CODIGO ES PARA ELIMINAR LAS IMAGENES CA QUE NO CORRESPONDEN A LA CARPETA AB
        # Delete file whose name starts with string 'pro'
        import glob
        pattern = r"/content/gdrive/MyDrive/DATASET/carpetaAB/CA*"
        for item in glob.iglob(pattern, recursive=True):
            os.remove(item)

        pattern = r"/content/gdrive/MyDrive/DATASET/carpetaAB/AC*"
        for item in glob.iglob(pattern, recursive=True):
            os.remove(item)

        #ESTE CODIGO ES PARA ELIMINAR LAS IMAGENES LA QUE NO CORRESPONDEN A LA CARPETA AB
        # Delete file whose name starts with string 'pro'
        pattern = r"/content/drive/gMyDrive/DATASET/carpetaAB/LA*"
        for item in glob.iglob(pattern, recursive=True):
            os.remove(item)

        #ARMAS CORTAS
        ##########################

        source_dir = "/content/gdrive/MyDrive/DATASET/UNIQUENESS_FINE_DATA"
        destination_dir = "/content/gdrive/MyDrive/DATASET/carpetaAC"
        shutil.copytree(source_dir, destination_dir)

        # Delete file whose name starts with string 'pro'
        pattern = r"/content/gdrive/MyDrive/DATASET/carpetaAC/AB*"
        for item in glob.iglob(pattern, recursive=True):
            os.remove(item)

        # Delete file whose name starts with string 'pro'
        pattern = r"/content/gdrive/MyDrive/DATASET/carpetaAC/LA*"
        for item in glob.iglob(pattern, recursive=True):
            os.remove(item)

        #ARMAS LARGAS
        ##########################

        source_dir = "/content/gdrive/MyDrive/DATASET/UNIQUENESS_FINE_DATA"
        destination_dir = "/content/gdrive/MyDrive/DATASET/carpetaAL"
        shutil.copytree(source_dir, destination_dir)

        # Delete file whose name starts with string 'pro'
        pattern = r"/content/gdrive/MyDrive/DATASET/carpetaAL/CA*"
        for item in glob.iglob(pattern, recursive=True):
            os.remove(item)

        pattern = r"/content/gdrive/MyDrive/DATASET/carpetaAB/AC*"
        for item in glob.iglob(pattern, recursive=True):
            os.remove(item)

        # Delete file whose name starts with string 'pro'
        pattern = r"/content/gdrive/MyDrive/DATASET/carpetaAL/AB*"
        for item in glob.iglob(pattern, recursive=True):
            os.remove(item)

        #REZIZING
        # Creación del array de armas de corto alcance
        armas_folder_path="/content/gdrive/MyDrive/DATASET/carpetaAC"
        armas_corto_alcance=[]
        
        for img in os.listdir(armas_folder_path):
            img = cv2.imread(os.path.join(armas_folder_path,img))
            img_gray= cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            img_resize= cv2.resize(img,(img_size,img_size))
            armas_corto_alcance.append(img_resize)
        armas_corto_alcance = np.array(armas_corto_alcance)
        print(armas_corto_alcance.shape)
      
        # Creación del array de armas de largo alcance
        armas_folder_path="/content/gdrive/MyDrive/DATASET/carpetaAL"
        armas_largo_alcance=[]
        
        for img in os.listdir(armas_folder_path):
            img = cv2.imread(os.path.join(armas_folder_path,img))
            img_gray= cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            img_resize= cv2.resize(img,(img_size,img_size))
            armas_largo_alcance.append(img_resize)
        armas_largo_alcance = np.array(armas_largo_alcance)
        print(armas_largo_alcance.shape)

        # Creación del array de armas blancas
        armas_folder_path="/content/gdrive/MyDrive/DATASET/carpetaAB"
        armas_blancas=[]
        
        for img in os.listdir(armas_folder_path):
            img = cv2.imread(os.path.join(armas_folder_path,img))
            img_gray= cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            img_resize= cv2.resize(img,(img_size,img_size))
            armas_blancas.append(img_resize)
        armas_blancas = np.array(armas_blancas)
        print(armas_blancas.shape)

        images=np.concatenate([armas_corto_alcance,armas_largo_alcance,armas_blancas])
        print(len(images))
        self.Images=np.array(images)
        print(self.Images.shape)

        etiquetas_ALA=np.repeat(0,len(armas_largo_alcance))
        print(len(etiquetas_ALA))

        etiquetas_ACA=np.repeat(1,len(armas_corto_alcance))
        print(len(etiquetas_ACA))

        etiquetas_AB=np.repeat(2,len(armas_blancas))
        print(len(etiquetas_AB))

        class_names=['Armas largo alcance','Armas corto alcance', 'Armas blancas' ]
        labels = np.concatenate([etiquetas_ALA, etiquetas_ACA, etiquetas_AB])
        print(len(labels))
        print(labels)
        self.Labels = np.array(labels)
        print(self.Labels.shape)

    def separardata(self):

        #Creación de los grupos de entrenamiento y testing para AC
        data = "/content/gdrive/MyDrive/DATASET/carpetaAC/"
        import os
        ldseg=np.array(os.listdir(data))
        i=0
        CAsize = len(ldseg)
        for filename in ldseg:
          o =i+1
          dst = str(o)+"CA"
          src =data + filename
          dst =data + dst
          os.rename(src, dst)
          i=i+1
        
        #training 70%
        #testing 10%
        #validation 20%
        intTest = int(CAsize*0.10)
        intVal = 2*intTest
        intTrain = CAsize - intVal - intTest #con esto garantizamos que las imágenes de training nunca sean menores al 70%

        source_dir = "/content/gdrive/MyDrive/DATASET/carpetaAC"
        destination_dir = "/content/gdrive/MyDrive/DATASET/TRAINAC"
        shutil.copytree(source_dir, destination_dir)

        destination_dir = "/content/gdrive/MyDrive/DATASET/TESTAC"
        shutil.copytree(source_dir, destination_dir)

        destination_dir = "/content/gdrive/MyDrive/DATASET/VALIDNAC"
        shutil.copytree(source_dir, destination_dir)

        data = "/content/gdrive/MyDrive/DATASET/TRAINAC/"

        for i in range(0, intTest+intVal):
            os.remove(data+str(i+1)+"CA") #SE BORRA EL PRIMER 30% DE DATA PARA QUEDARSE CON EL 70% RESTANTE

        data = "/content/gdrive/MyDrive/DATASET/TESTAC/"

        for i in range(0, intVal):
            os.remove(data+str(i+1)+"CA") #SE BORRA 20% DE LAS PRIMERA IÁGENES, 
        for i in range(intTest+intVal, CAsize):
            os.remove(data+str(i+1)+"CA") #SE DEJA EL OTRO 10% Y SE BORRA EL RESTO PARA QUEDARSE CON EL 10% RESTANTE   

        data = "/content/gdrive/MyDrive/DATASET/VALIDNAC/"

        for i in range(intVal, CAsize):
            os.remove(data+str(i+1)+"CA") #SE DEJA EL PRIMER 20% DE IMÁGENES Y SE BORRA EL RESTO DE IMÁGENES
              
        #Creación de los grupos de entrenamiento y testing para AB
        data = "/content/gdrive/MyDrive/DATASET/carpetaAB/"
        import os
        ldseg=np.array(os.listdir(data))
        i=0
        CAsize = len(ldseg)
        for filename in ldseg:
          o =i+1
          dst = str(o)+"AB"
          src =data + filename
          dst =data + dst
          os.rename(src, dst)
          i=i+1

        #training 70%
        #testing 10%
        #validation 20%
        intTest = int(CAsize*0.10)
        intVal = 2*intTest
        intTrain = CAsize - intVal - intTest #con esto garantizamos que las imágenes de training nunca sean menores al 70%    

        source_dir = "/content/gdrive/MyDrive/DATASET/carpetaAB"
        destination_dir = "/content/gdrive/MyDrive/DATASET/TRAINAB"
        shutil.copytree(source_dir, destination_dir)

        destination_dir = "/content/gdrive/MyDrive/DATASET/TESTAB"
        shutil.copytree(source_dir, destination_dir)
        
        destination_dir = "/content/gdrive/MyDrive/DATASET/VALIDNAB"
        shutil.copytree(source_dir, destination_dir)

        data = "/content/gdrive/MyDrive/DATASET/TRAINAB/"

        for i in range(0, intTest+intVal):
            os.remove(data+str(i+1)+"AB") #SE BORRA EL PRIMER 30% DE DATA PARA QUEDARSE CON EL 70% RESTANTE

        data = "/content/gdrive/MyDrive/DATASET/TESTAB/"

        for i in range(0, intVal):
            os.remove(data+str(i+1)+"AB") #SE BORRA 20% DE LAS PRIMERA IÁGENES, 
        for i in range(intTest+intVal, CAsize):
            os.remove(data+str(i+1)+"AB") #SE DEJA EL OTRO 10% Y SE BORRA EL RESTO PARA QUEDARSE CON EL 10% RESTANTE   

        data = "/content/gdrive/MyDrive/DATASET/VALIDNAB/"

        for i in range(intVal, CAsize):
            os.remove(data+str(i+1)+"AB") #SE DEJA EL PRIMER 20% DE IMÁGENES Y SE BORRA EL RESTO DE IMÁGENES

        #Creación de los grupos de entrenamiento y testing para AL
        data = "/content/gdrive/MyDrive/DATASET/carpetaAL/"
        import os
        ldseg=np.array(os.listdir(data))
        i=0
        CAsize = len(ldseg)
        for filename in ldseg:
          o =i+1
          dst = str(o)+"AL"
          src =data + filename
          dst =data + dst
          os.rename(src, dst)
          i=i+1

        #training 70%
        #testing 10%
        #validation 20%
        intTest = int(CAsize*0.10)
        intVal = 2*intTest
        intTrain = CAsize - intVal - intTest #con esto garantizamos que las imágenes de training nunca sean menores al 70%

        source_dir = "/content/gdrive/MyDrive/DATASET/carpetaAL"
        destination_dir = "/content/gdrive/MyDrive/DATASET/TRAINAL"
        shutil.copytree(source_dir, destination_dir)

        destination_dir = "/content/gdrive/MyDrive/DATASET/TESTAL"
        shutil.copytree(source_dir, destination_dir)
        
        destination_dir = "/content/gdrive/MyDrive/DATASET/VALIDNAL"
        shutil.copytree(source_dir, destination_dir)

        data = "/content/gdrive/MyDrive/DATASET/TRAINAL/"

        for i in range(0, intTest+intVal):
            os.remove(data+str(i+1)+"AL") #SE BORRA EL PRIMER 30% DE DATA PARA QUEDARSE CON EL 70% RESTANTE

        data = "/content/gdrive/MyDrive/DATASET/TESTAL/"

        for i in range(0, intVal):
            os.remove(data+str(i+1)+"AL") #SE BORRA 20% DE LAS PRIMERA IÁGENES, 
        for i in range(intTest+intVal, CAsize):
            os.remove(data+str(i+1)+"AL") #SE DEJA EL OTRO 10% Y SE BORRA EL RESTO PARA QUEDARSE CON EL 10% RESTANTE   

        data = "/content/gdrive/MyDrive/DATASET/VALIDNAL/"

        for i in range(intVal, CAsize):
            os.remove(data+str(i+1)+"AL") #SE DEJA EL PRIMER 20% DE IMÁGENES Y SE BORRA EL RESTO DE IMÁGENES
                    

Migrating database to v0.18.0


INFO:fiftyone.migrations.runner:Migrating database to v0.18.0
